# filtering

> Filtering, grouping, and keyboard navigation route handlers

In [ ]:
#| default_exp routes.filtering

In [ ]:
#| export
from typing import Tuple, Dict, Callable

from fasthtml.common import APIRouter

from cjm_fasthtml_interactions.core.state_store import get_session_id

from cjm_transcript_source_select.models import SelectionUrls
from cjm_transcript_source_select.routes.core import (
    WorkflowStateStore, _get_step_state, _update_step_state, _build_queue_response,
    _check_duplicate_media_path
)
from cjm_transcript_source_select.components.source_browser import (
    _render_source_list
)
from cjm_transcript_source_select.services.source import SourceService
from cjm_transcript_source_select.services.source_utils import (
    filter_transcriptions, toggle_source_selection, reorder_item, is_source_selected
)

## Filter Handler

In [ ]:
#| export
def _handle_source_filter(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    search: str,  # Search term from input
    urls: SelectionUrls,  # URL bundle for rendering
):  # Filtered source list component
    """Filter transcription sources by search term."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    all_transcriptions = source_service.query_transcriptions(limit=500)
    filtered = filter_transcriptions(all_transcriptions, search)
    
    return _render_source_list(
        transcriptions=filtered,
        selected_sources=selected_sources,
        add_url=urls.add,
        remove_url=urls.remove,
        preview_url=urls.preview,
        select_all_url=urls.select_all,
    )

## Grouping Handler

In [ ]:
#| export
def _handle_grouping_change(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    grouping_mode: str,  # New grouping mode: "media_path" or "batch_id"
    urls: SelectionUrls,  # URL bundle for rendering
):  # Updated source list component
    """Change the grouping mode and re-render the source list."""
    session_id = get_session_id(sess)
    # Save the new grouping mode to state
    _update_step_state(state_store, workflow_id, session_id, grouping_mode=grouping_mode)
    
    # Get current state for rendering
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    # Get all transcriptions
    all_transcriptions = source_service.query_transcriptions(limit=500)
    
    # Render source list with new grouping mode
    return _render_source_list(
        transcriptions=all_transcriptions,
        selected_sources=selected_sources,
        add_url=urls.add,
        remove_url=urls.remove,
        preview_url=urls.preview,
        select_all_url=urls.select_all,
        grouping_mode=grouping_mode,
    )

## Keyboard Handlers

In [ ]:
#| export
def _handle_selection_toggle_focused(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    record_id: str,  # Job ID from focused row (via hx-include)
    provider_id: str,  # Plugin name from focused row (via hx-include)
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component with OOB stats, optionally with OOB source list
    """Toggle selection of the focused row (keyboard shortcut handler)."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    # Only check for duplicate media_path when adding (not removing)
    if not is_source_selected(record_id, provider_id, selected_sources):
        if _check_duplicate_media_path(source_service, record_id, provider_id, selected_sources):
            return _build_queue_response(
                state_store, workflow_id, source_service, session_id, selected_sources, urls
            )
    
    selected_sources = toggle_source_selection(record_id, provider_id, selected_sources)
    _update_step_state(state_store, workflow_id, session_id, selected_sources)
    
    return _build_queue_response(state_store, workflow_id, source_service, session_id, selected_sources, urls)

In [ ]:
#| export
def _handle_keyboard_reorder(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    request,  # FastHTML request object
    sess,  # FastHTML session object
    record_id: str,  # Record ID of item to move
    provider_id: str,  # Provider ID of item to move
    direction: str,  # Direction to move: "up" or "down"
    urls: SelectionUrls,  # URL bundle for rendering
):  # Queue component, optionally with OOB source list
    """Move an item up or down in the selection queue via keyboard."""
    session_id = get_session_id(sess)
    step_state = _get_step_state(state_store, workflow_id, session_id)
    selected_sources = step_state.get("selected_sources", [])
    
    selected_sources = reorder_item(selected_sources, record_id, provider_id, direction)
    _update_step_state(state_store, workflow_id, session_id, selected_sources)
    
    return _build_queue_response(
        state_store, workflow_id, source_service, session_id, selected_sources, urls, include_stats=False,
    )

## Router

In [ ]:
#| export
def init_filtering_router(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    source_service: SourceService,  # The source service for queries
    prefix: str,  # Route prefix (e.g., "/workflow/selection/filtering")
    urls: SelectionUrls,  # URL bundle for rendering
) -> Tuple[APIRouter, Dict[str, Callable]]:  # (router, route_dict)
    """Initialize filtering and keyboard navigation routes."""
    router = APIRouter(prefix=prefix)

    @router
    def toggle_focused(request, sess, record_id: str, provider_id: str):
        """Toggle selection of the focused row (keyboard shortcut)."""
        return _handle_selection_toggle_focused(
            state_store, workflow_id, source_service,
            request, sess, record_id, provider_id, urls=urls,
        )

    @router
    def keyboard_reorder(request, sess, record_id: str, provider_id: str, direction: str):
        """Move an item up or down in the queue via keyboard (Shift+Up/Down)."""
        return _handle_keyboard_reorder(
            state_store, workflow_id, source_service,
            request, sess, record_id, provider_id, direction, urls=urls,
        )

    @router
    def filter(request, sess, search: str = ""):
        """Filter source list by search term."""
        return _handle_source_filter(
            state_store, workflow_id, source_service,
            request, sess, search, urls=urls,
        )

    @router
    def grouping_change(request, sess, grouping_mode: str):
        """Change the grouping mode for the source list."""
        return _handle_grouping_change(
            state_store, workflow_id, source_service,
            request, sess, grouping_mode, urls=urls,
        )

    routes = {
        "toggle_focused": toggle_focused,
        "keyboard_reorder": keyboard_reorder,
        "filter": filter,
        "grouping_change": grouping_change,
    }

    return router, routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()